In [1]:
%matplotlib inline
from collections import defaultdict
import csv
import os
import sys

import matplotlib.pyplot
import cv2
from shapely.geometry import MultiPolygon, Polygon
import shapely.wkt
import shapely.affinity
import numpy as np
import tifffile as tiff
import time
import pandas as pd
np.random.seed(42)

csv.field_size_limit(sys.maxsize);
cur_dir = '/home/rob/Udacity/capstone/data'

print cv2.__version__

from sklearn.linear_model import SGDClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import average_precision_score
from sklearn.metrics import recall_score
from sklearn.utils import shuffle

3.2.0-dev


In [2]:
IM_ID = '6120_2_2'
ClassNames = {'1':'buildings', '2':'Misc. Manmade structures', '3': 'Road', '4':'Track', '5':'Trees',
                    '6':'Crops', '7':'Waterway', '8':'Standing water', '9':'Vehicle Large ', '10':'Vehicle Small'}

save_masks = False


def get_scalers(im_size):
    h, w = im_size  # they are flipped so that mask_for_polygons works correctly
    h, w = float(h), float(w)
    w_ = w * (w / (w + 1))
    h_ = h * (h / (h + 1))
    return w_ / x_max, h_ / y_min

def mask_for_polygons(polygons):
    img_mask = np.zeros(im_size, np.uint8)
    if not polygons:
        return img_mask
    int_coords = lambda x: np.array(x).round().astype(np.int32)
    exteriors = [int_coords(poly.exterior.coords) for poly in polygons]
    interiors = [int_coords(pi.coords) for poly in polygons
                 for pi in poly.interiors]
    cv2.fillPoly(img_mask, exteriors, 1)
    cv2.fillPoly(img_mask, interiors, 0)
    return img_mask

def scale_percentile(matrix):
    w, h, d = matrix.shape
    matrix = np.reshape(matrix, [w * h, d]).astype(np.float64)
    # Get 2nd and 98th percentile
    mins = np.percentile(matrix, 1, axis=0)
    maxs = np.percentile(matrix, 99, axis=0) - mins
    matrix = (matrix - mins[None, :]) / maxs[None, :]
    matrix = np.reshape(matrix, [w, h, d])
    matrix = matrix.clip(0, 1)
    return matrix

def show_mask(m):
        
    tiff.imshow(255 * np.stack([m, m, m]));
    
def mask_to_polygons(mask, epsilon=10., min_area=10.):
    # first, find contours with cv2: it's much faster than shapely
    image, contours, hierarchy = cv2.findContours(
        ((mask == 1) * 255).astype(np.uint8),
        cv2.RETR_CCOMP, cv2.CHAIN_APPROX_TC89_KCOS)
    # create approximate contours to have reasonable submission size
    approx_contours = [cv2.approxPolyDP(cnt, epsilon, True)
                       for cnt in contours]
    if not contours:
        return MultiPolygon()
    # now messy stuff to associate parent and child contours
    cnt_children = defaultdict(list)
    child_contours = set()
    assert hierarchy.shape[0] == 1
    # http://docs.opencv.org/3.1.0/d9/d8b/tutorial_py_contours_hierarchy.html
    for idx, (_, _, _, parent_idx) in enumerate(hierarchy[0]):
        if parent_idx != -1:
            child_contours.add(idx)
            cnt_children[parent_idx].append(approx_contours[idx])
    # create actual polygons filtering by area (removes artifacts)
    all_polygons = []
    for idx, cnt in enumerate(approx_contours):
        if idx not in child_contours and cv2.contourArea(cnt) >= min_area:
            assert cnt.shape[1] == 1
            poly = Polygon(
                shell=cnt[:, 0, :],
                holes=[c[:, 0, :] for c in cnt_children.get(idx, [])
                       if cv2.contourArea(c) >= min_area])
            all_polygons.append(poly)
    # approximating polygons might have created invalid ones, fix them
    all_polygons = MultiPolygon(all_polygons)
    if not all_polygons.is_valid:
        all_polygons = all_polygons.buffer(0)
        # Sometimes buffer() converts a simple Multipolygon to just a Polygon,
        # need to keep it a Multi throughout
        if all_polygons.type == 'Polygon':
            all_polygons = MultiPolygon([all_polygons])
    return all_polygons

def partial_pipe_fit(pipeline_obj, xs, ys):
    xs = pipeline_obj.named_steps['standardscaler'].fit_transform(xs)
    pipeline_obj.named_steps['sgdclassifier'].partial_fit(xs,ys,[0,1])


class MY_SGDClassifier(SGDClassifier):
    def __init__(self, n_jobs=3, n_iter=1, loss='log'):
        super(MY_SGDClassifier, self).__init__(loss='log', penalty='l2', alpha=0.0001, 
                                               l1_ratio=0.15, fit_intercept=True, n_iter=n_iter, shuffle=True, 
                                               verbose=0, epsilon=0.1, n_jobs=n_jobs, random_state=None, 
                                               learning_rate='optimal', eta0=0.0, power_t=0.5, class_weight=None, 
                                               warm_start=False, average=False)
    def fit(self, X, y, classes=[0,1], sample_weight=None):
        self.partial_fit(X,y,classes)
    

In [3]:
classifiers = {}
train_masks = {}
pred_masks = {}

trainIM_IDs = []
with open(cur_dir + '/train_wkt_v4.csv') as csvfile:
    reader = csv.reader(csvfile, delimiter=',', quotechar=',')
    for i,row in enumerate(reader):
        if i == 0:
            i = 1
        if (i%10) == 0:
            trainIM_IDs.append(row[0])
            
testIM_IDs = []
with open(cur_dir + '/sample_submission.csv') as csvfile:
    reader = csv.reader(csvfile, delimiter=',', quotechar=',')
    for i,row in enumerate(reader):
        if i == 0:
            i = 1
        if (i%10) == 0:
            testIM_IDs.append(row[0])
            

In [ ]:
viewPictures = False

def show_picture(IM_ID):
    im_rgb = tiff.imread('../capstone/data/three_band/{}.tif'.format(IM_ID)).transpose([1, 2, 0])
    tiff.imshow(255 * scale_percentile(im_rgb[:,:]))

if viewPictures:
    for IM_ID in trainIM_IDs:
        show_picture(IM_ID)

In [ ]:
### Fitting the standardscaler to all data ###

all_pictures = trainIM_IDs + testIM_IDs
scaler = StandardScaler()

for IM_ID in all_pictures:
    
    im_rgb = tiff.imread(cur_dir +'/three_band/{}.tif'.format(IM_ID)).transpose([1, 2, 0]) #gives back lists of lists
    xs = im_rgb.reshape(-1, 3).astype(np.float32)
    scaler.partial_fit(xs)

mean = scaler.mean_
std = scaler.var_
scale = scaler.scale_
samples_seen = scaler.n_samples_seen_
a = scaler.get_params()

print mean, std, scale, samples_seen

In [ ]:
### Training SGD on Data


counter = 1
train_set = trainIM_IDs[1:] + [trainIM_IDs[0]]
#train_set = trainIM_IDs[1:2] + trainIM_IDs[3:12] + trainIM_IDs[13:16] + trainIM_IDs[17:19] + trainIM_IDs[20:]
test_set = [trainIM_IDs[0],trainIM_IDs[2],trainIM_IDs[12], trainIM_IDs[16], trainIM_IDs[19]]

trainDataLength = len(train_set)

(l1,l2,l3,l4,l5) = [],[],[],[],[]

timer = time.time()


for IM_ID in train_set:
    print IM_ID
    print "{}. picture of {}".format(counter, trainDataLength)
    
    x_max = y_min = None
    for _im_id, _x, _y in csv.reader(open(cur_dir + '/grid_sizes.csv')):
        if _im_id == IM_ID:
            x_max, y_min = float(_x), float(_y)
            break
    
    
    for i in range(1,11): #range(1,11)
        POLY_TYPE = str(i)
        print ClassNames[POLY_TYPE]
        l1.append(IM_ID)
        l2.append(ClassNames[POLY_TYPE])

        # Load train poly with shapely
        train_polygons = None
        for _im_id, _poly_type, _poly in csv.reader(open(cur_dir + '/train_wkt_v4.csv')):
            if _im_id == IM_ID and _poly_type == POLY_TYPE:
                train_polygons = shapely.wkt.loads(_poly)
                break
                
        im_rgb = tiff.imread(cur_dir +'/three_band/{}.tif'.format(IM_ID)).transpose([1, 2, 0])
        im_size = im_rgb.shape[:2]
        
        """
        # Read image with tiff
        im_rgb = tiff.imread(cur_dir +'/sixteen_band/{}_M.tif'.format(IM_ID)).transpose([1, 2, 0])
        im_size = im_rgb.shape[:2]
        img = np.zeros((im_rgb.shape[0],im_rgb.shape[1],3))
        img[:,:,0] = im_rgb[:,:,7] #red
        img[:,:,1] = im_rgb[:,:,4] #green
        img[:,:,2] = im_rgb[:,:,0] #blue
        im_rgb = img
        """
        x_scaler, y_scaler = get_scalers(im_size)

        train_polygons_scaled = shapely.affinity.scale(
            train_polygons, xfact=x_scaler, yfact=y_scaler, origin=(0, 0, 0))

        train_mask = mask_for_polygons(train_polygons_scaled)
        
        xs = im_rgb.reshape(-1, 3).astype(np.float32)
        ys = train_mask.reshape(-1)
        xs = scaler.transform(xs)
        
        """Initialize pipeline"""
        if IM_ID == train_set[0]:
            #pipeline = make_pipeline(StandardScaler(), SGDClassifier(loss='log',n_iter=1))
            pipe2 = MY_SGDClassifier(loss='log',n_iter=1)
        else:
            pipe2 = classifiers["SGD{}".format(ClassNames[POLY_TYPE])]

        print('training...')
        
        #partial_pipe_fit(pipeline, xs, ys)
        #pred_ys = pipeline.predict_proba(xs)[:, 1]
        
        try:
            pipe2 = pipe2.partial_fit(xs,ys, [0,1])
            pred_ys = (pipe2.predict_proba(xs).T)[1]
        except ValueError:
            "Class not represented"
            classifiers["SGD{}".format(ClassNames[POLY_TYPE])] = pipe2
            continue
        pred_mask = pred_ys.reshape(train_mask.shape)
        threshold = 0.30
        pred_binary_mask = pred_mask >= threshold

        #Save all for later use
        if save_masks:
            pred_masks["SGD{}".format(ClassNames[POLY_TYPE])] = pred_mask
            train_masks["SGD{}".format(ClassNames[POLY_TYPE])] = train_mask

        classifiers["SGD{}".format(ClassNames[POLY_TYPE])] = pipe2

        pred_polygons = mask_to_polygons(pred_binary_mask,epsilon=10., min_area=10.)
        pred_poly_mask = mask_for_polygons(pred_polygons)

        scaled_pred_polygons = shapely.affinity.scale(
            pred_polygons, xfact=1 / x_scaler, yfact=1 / y_scaler, origin=(0, 0, 0))

        dumped_prediction = shapely.wkt.dumps(scaled_pred_polygons) #This is exactly the wanted output for submission
        final_polygons = shapely.wkt.loads(dumped_prediction)

        if final_polygons == "GEOMETRYCOLLECTION EMPTY":
            print "Algorithm did not detect this class or it is not in dataset"
        else:
            tp, fp, fn = (( pred_binary_mask &  train_mask).sum(),
                      ( pred_binary_mask & ~train_mask).sum(),
                      (~pred_binary_mask &  train_mask).sum())
            avg_prec_score = average_precision_score(ys, pred_ys)
            avg_recall = recall_score(ys, pred_binary_mask.reshape(-1))
            print('average precision (correct prediction(tp)/ all predicions, true or false (tp+fp))', avg_prec_score)
            print('recall_score (correct prediction(tp)/ real amount of that class(tp+fn)) ', avg_recall)
            pixel_jaccard = (float(tp) / (tp + fp + fn))
            print('Pixel jaccard', pixel_jaccard)
            print('Prediction size: {:,} bytes'.format(len(dumped_prediction)))
            l3.append(avg_recall)
            l4.append(pixel_jaccard)
            try:
                final_jaccard = (final_polygons.intersection(train_polygons).area /
                      final_polygons.union(train_polygons).area)
                print('Final jaccard',final_jaccard)
                l5.append(final_jaccard)
            except ZeroDivisionError:
                print "Warning: Error in Final jaccard calculation!"
        print ""
        print ""
        
        
    counter += 1
timer_end = time.time()
print (timer_end - timer)/60

In [ ]:
records = pd.DataFrame([l1,l2,l3,l4,l5],index=['IM_ID','CLASS','PRECI','PIXEL JAC','FINAL JAC']).T
print records.describe()

In [ ]:
###  --- Test the train result on validation data. This is not used right now --- ###

if False:
    for IM_ID in test_set:
        print IM_ID
        print "{}. picture".format(counter)

        x_max = y_min = None
        for _im_id, _x, _y in csv.reader(open(cur_dir + '/grid_sizes.csv')):
            if _im_id == IM_ID:
                x_max, y_min = float(_x), float(_y)
                break


        for i in range(1,11): #range(1,11)
            POLY_TYPE = str(i)
            print ClassNames[POLY_TYPE]
            l1.append(IM_ID)
            l2.append(ClassNames[POLY_TYPE])

            im_rgb = tiff.imread(cur_dir +'/three_band/{}.tif'.format(IM_ID)).transpose([1, 2, 0])
            im_size = im_rgb.shape[:2]
            """
            # Read image with tiff
            im_rgb = tiff.imread(cur_dir +'/sixteen_band/{}_M.tif'.format(IM_ID)).transpose([1, 2, 0])
            im_size = im_rgb.shape[:2]
            img = np.zeros((im_rgb.shape[0],im_rgb.shape[1],3))
            img[:,:,0] = im_rgb[:,:,7] #red
            img[:,:,1] = im_rgb[:,:,4] #green
            img[:,:,2] = im_rgb[:,:,0] #blue
            im_rgb = img
            """

            x_scaler, y_scaler = get_scalers()

            train_polygons_scaled = shapely.affinity.scale(
                train_polygons, xfact=x_scaler, yfact=y_scaler, origin=(0, 0, 0))


            train_mask = mask_for_polygons(train_polygons_scaled)

            xs = im_rgb.reshape(-1, 3).astype(np.float32)
            ys = train_mask.reshape(-1)


            """Benutze train pipeline"""

            pipeline = classifiers["SGD{}".format(ClassNames[POLY_TYPE])]

            print('testing...')

            pred_ys = pipeline.predict_proba(xs)[:, 1]
            pred_mask = pred_ys.reshape(train_mask.shape)

            threshold = 0.31
            pred_binary_mask = pred_mask >= threshold

            #Save all for later use
            if save_masks:
                pred_masks["SGD{}".format(ClassNames[POLY_TYPE])] = pred_mask
                train_masks["SGD{}".format(ClassNames[POLY_TYPE])] = train_mask

            pred_polygons = mask_to_polygons(pred_binary_mask)
            pred_poly_mask = mask_for_polygons(pred_polygons)

            scaled_pred_polygons = shapely.affinity.scale(
                pred_polygons, xfact=1 / x_scaler, yfact=1 / y_scaler, origin=(0, 0, 0))

            dumped_prediction = shapely.wkt.dumps(scaled_pred_polygons) #This is exactly the wanted output for submission
            final_polygons = shapely.wkt.loads(dumped_prediction)

            if dumped_prediction == "GEOMETRYCOLLECTION EMPTY":
                print "No pixels predicted by algorithm"
            else:
                tp, fp, fn = (( pred_binary_mask &  train_mask).sum(),
                          ( pred_binary_mask & ~train_mask).sum(),
                          (~pred_binary_mask &  train_mask).sum())
                print('average precision', average_precision_score(ys, pred_ys))
                pixel_jaccard = (float(tp) / (tp + fp + fn))
                print('Pixel jaccard', pixel_jaccard)
                print('Prediction size: {:,} bytes'.format(len(dumped_prediction)))
                l3.append(average_precision_score)
                l4.append(pixel_jaccard)
                try:
                    final_jaccard = (final_polygons.intersection(train_polygons).area /
                          final_polygons.union(train_polygons).area)
                    print('Final jaccard',final_jaccard)
                    l5.append(final_jaccard)
                except TopologicalError:
                    print "Warning: Error in Final jaccard calculation!"
            print ""
            print ""

    pd.DataFrame([l1,l2,l3,l4,l5],index=['IM_ID','CLASS','PRECI','PIXEL JAC','FINAL JAC']).T

In [ ]:
if save_masks:
    for i in range(1,11):
        POLY_TYPE = str(i)
        show_mask(pred_masks["SGD{}".format(ClassNames[POLY_TYPE])][:,:])
        show_mask(train_masks["SGD{}".format(ClassNames[POLY_TYPE])][:,:])
        #show_mask(pred_mask[:,:])
    


In [ ]:
#get picture ID

train_polygons = None

big_timer = time.time()
predictions = []
picture_counter = 1
for IM_ID in testIM_IDs:
    print IM_ID
    print "{}. picture out of {}".format(picture_counter, len(testIM_IDs))
    picture_counter += 1
    start = time.time()
    
    x_max = y_min = None
    for _im_id, _x, _y in csv.reader(open(cur_dir + '/grid_sizes.csv')):
        if _im_id == IM_ID:
            x_max, y_min = float(_x), float(_y)
            break
    
    
    for i in range(1,11):
        POLY_TYPE = str(i)

        # Read image with tiff
        im_rgb = tiff.imread(cur_dir +'/three_band/{}.tif'.format(IM_ID)).transpose([1, 2, 0])
        im_size = im_rgb.shape[:2]

        x_scaler, y_scaler = get_scalers(im_size)

        xs = im_rgb.reshape(-1, 3).astype(np.float32)

        pipe2 = classifiers["SGD{}".format(ClassNames[POLY_TYPE])]
        xs = scaler.transform(xs)
        try:
            pred_ys = pipe2.predict(xs) #pipeline.predict_proba(xs)[:, 1]
        except IndexError:
            print ("No prediction possible. Instead prediction mask will"
                   "be zeros only")
            pred_ys = np.zeros(im_size)

        pred_mask = pred_ys.reshape(im_size)

        threshold = 0.30
        pred_binary_mask = pred_mask >= threshold
        pred_polygons = mask_to_polygons(pred_binary_mask, epsilon=10., min_area=10.)
        scaled_pred_polygons = shapely.affinity.scale(
            pred_polygons, xfact=1 / x_scaler, yfact=1 / y_scaler, origin=(0, 0, 0))

        dumped_prediction = shapely.wkt.dumps(scaled_pred_polygons)
        if dumped_prediction == 'GEOMETRYCOLLECTION EMPTY':
            dumped_prediction = 'MULTIPOLYGON EMPTY'

        predictions.append([IM_ID,POLY_TYPE, dumped_prediction])
        
    end = time.time()
    print "Prediction took",(end-start), "seconds"
    
big_timer_end = time.time()
print "Prediction time for all pictures in Minutes: ", (big_timer_end- big_timer)/60

In [ ]:
with open("predictions6.csv","wb") as csvfile:
    writer = csv.writer(csvfile, delimiter=',')
    writer.writerow(['ImageId','ClassType','MultipolygonWKT'])
    for prediction in predictions:
        writer.writerow(prediction)
